# Figure 6

Assessment of cloud water path (CWP) detectability and falsely-detected CWP.

In [ ]:
import os
from string import ascii_lowercase as abc

import cmcrameri.cm as cmc
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import xarray as xr
from dotenv import load_dotenv
from lizard import ac3airlib
from lizard.mpltools import style
from lizard.readers.band_pass import read_band_pass
from lizard.writers.figure_to_file import write_figure
from scipy.optimize import curve_fit

from si_clouds.helpers.sigmoid import sigmoid
from si_clouds.io.readers.ancillary import read_ancillary_data
from si_clouds.io.readers.oem_result import read_oem_result_concat
from si_clouds.io.readers.velox_class import read_velox_classification
from si_clouds.io.readers.yif_cwp_iwv_sensitivity import read_yif_cwp_sensitivity

load_dotenv()

In [ ]:
ds_bp = read_band_pass("HAMP")
ds_anc = read_ancillary_data()

In [ ]:
# velox surface classification
ds_cla = read_velox_classification()
ds_cla = xr.concat(
    [ds_cla.hamp0, ds_cla.hamp1, ds_cla.hamp2, ds_cla.hamp3], dim="angle"
)
ds_cla["angle"] = [1.3, 1.65, 1.75, 2.5]

# drop data from HALO-AC3_HALO_RF04_hl08, as the thin ice fraction is always 1
ds_cla = ds_cla.sel(
    time=(ds_cla.time < ac3airlib.segment_times("HALO-AC3_HALO_RF04_hl08")[0])
    | (ds_cla.time > ac3airlib.segment_times("HALO-AC3_HALO_RF04_hl08")[1])
)

In [ ]:
ds_a_fu, ds_op_fu, _, _ = read_oem_result_concat(
    version="pub_r2_clearsky_v1", test_id="", write=False
)

# remove times where the retrieval was not valid
ds_op_fu = ds_op_fu.sel(time=ds_anc.ix_retrieval_valid.sel(time=ds_op_fu.time))

# for young ice fraction jacobian
ds_sen = read_yif_cwp_sensitivity()
da_sen_cwp = ds_sen.y1 - ds_sen.y0
da_sen_yif = ds_sen.y2 - ds_sen.y0

In [ ]:
step = 20
bins_dist = np.arange(0, 1001, step) * 1e3
bins_center = (bins_dist[:-1] + bins_dist[1:]) / 2
da_cwp_95_sic = ds_op_fu.cwp.groupby_bins(
    ds_anc.dist_sic_0_50.sel(time=ds_op_fu.time), bins=bins_dist
).quantile(0.95)
da_cwp_95_sic_count = ds_op_fu.cwp.groupby_bins(
    ds_anc.dist_sic_0_50.sel(time=ds_op_fu.time), bins=bins_dist
).count()

popt, pcov = curve_fit(
    sigmoid,
    bins_center[~np.isnan(da_cwp_95_sic.values)],
    da_cwp_95_sic.values[~np.isnan(da_cwp_95_sic.values)],
    p0=[-0.3, 0.0001, 150e3, 0.35],
)

# write optimal parameters to text file for other routines
file = os.path.join(
    os.environ["PATH_SEC"],
    "data/sea_ice_clouds/cwp_sensitivity",
    "cwp_sensitivity.npy",
)
with open(file, "wb") as f:
    np.save(f, popt)

In [ ]:
fig, axes = plt.subplot_mosaic(
    [["cdf", "ice_edge"], ["thin_ice", "jacobian"]],
    figsize=(6, 5),
    layout="constrained",
)

for i, ax in enumerate(axes.values()):
    ax.annotate(
        f"({abc[i]})",
        xy=(0.98, 0.93),
        xycoords="axes fraction",
        ha="right",
        va="top",
    )

# add spines to all axis
for ax in axes.values():
    for spine in ax.spines.values():
        spine.set_visible(True)

# cdf and detection thresholds
bins = np.arange(0, 1000, step)
axes["cdf"].hist(
    ds_op_fu.cwp * 1e3,
    bins=bins,
    histtype="step",
    cumulative=True,
    density=True,
    color="k",
    label="All",
    linewidth=plt.rcParams["lines.linewidth"],
)

dist_bins = [(200, 1e9)]
labels = ["Central Arctic"]
colors = [cmc.batlow(0.7)]
for i, (dist0, dist1) in enumerate(dist_bins):
    da_dist = ds_anc.dist_sic_0_50.sel(time=ds_op_fu.time)
    axes["cdf"].hist(
        ds_op_fu.cwp.where((da_dist >= dist0 * 1e3) & (da_dist < dist1 * 1e3))
        * 1e3,
        bins=bins,
        histtype="step",
        cumulative=True,
        density=True,
        color=colors[i],
        label=labels[i],
        linewidth=plt.rcParams["lines.linewidth"],
    )
    print(
        ds_op_fu.cwp.where((da_dist >= dist0 * 1e3) & (da_dist < dist1 * 1e3))
        .quantile(0.95)
        .item()
        * 1e3
    )
    axes["cdf"].axvline(
        ds_op_fu.cwp.where(
            (da_dist >= dist0 * 1e3) & (da_dist < dist1 * 1e3)
        ).quantile(0.95)
        * 1e3,
        color=colors[i],
        linestyle=":",
    )

axes["cdf"].axhline(
    0.95,
    color="k",
    zorder=-1,
    linestyle="--",
    label=r"95$^{\mathrm{th}}$ percentile",
)
print(ds_op_fu.cwp.quantile(0.95).item() * 1e3)
axes["cdf"].axvline(
    ds_op_fu.cwp.quantile(0.95) * 1e3,
    color="k",
    linestyle=":",
    label="CWP det.",
)

axes["cdf"].set_xlim(0, 500)
axes["cdf"].set_ylim(0, 1)

axes["cdf"].legend(loc="center right", frameon=True, bbox_to_anchor=(1, 0.4))

axes["cdf"].set_xlabel("Falsely-detected CWP [g m$^{-2}$]")
axes["cdf"].set_ylabel("Cumulative density")

# check the 95th percentile binned by distance to open water
axes["ice_edge"].plot(
    bins_center * 1e-3,
    da_cwp_95_sic * 1e3,
    zorder=1000,
    color=cmc.batlow(0.3),
    marker=".",
    label="CWP det.",
)

axes["ice_edge"].plot(
    np.arange(0, 1001, 50),
    sigmoid(np.arange(0, 1001, 50) * 1e3, *popt) * 1e3,
    color=cmc.batlow(0.3),
    label="CWP det. fit",
    linestyle=":",
)

ax_cnt = axes["ice_edge"].twinx()
ax_cnt.bar(
    bins_center * 1e-3,
    da_cwp_95_sic_count,
    width=step,
    color=cmc.batlow(0.7),
    zorder=-1,
    label="Count",
    linewidth=0.75,
    edgecolor="k",
)

axes["ice_edge"].set_zorder(2)
ax_cnt.set_zorder(-1)
axes["ice_edge"].patch.set_visible(False)

axes["ice_edge"].set_ylim(0, 400)
ax_cnt.set_ylim(0, 2000)

axes["ice_edge"].set_xlim(0, 800)
ax_cnt.set_xlim(0, 800)

handles1, labels1 = axes["ice_edge"].get_legend_handles_labels()
handles2, labels2 = ax_cnt.get_legend_handles_labels()
axes["ice_edge"].legend(
    handles1 + handles2,
    labels1 + labels2,
    loc="center right",
    frameon=False,
    bbox_to_anchor=(1, 0.6),
)

axes["ice_edge"].set_xlabel("Distance to ice edge [km]")
axes["ice_edge"].set_ylabel("CWP detectability [g m$^{-2}$]")
ax_cnt.set_ylabel("Count")

# falsely detected cwp vs. velox-based thin ice fraction
ds_op_aligned, ds_cla_aligned = xr.align(ds_op_fu, ds_cla)
x = ds_cla_aligned.sel(surface_type=3, angle=2.5) * 100
y = ds_op_aligned.cwp * 1e3
c = ds_op_aligned.y_obs.sel(channel=15) - ds_op_aligned.y_obs.sel(channel=1)
im = axes["thin_ice"].scatter(
    x, y, c=c, s=5, lw=0, cmap=cmc.batlow, vmin=-40, vmax=10
)

axes["thin_ice"].set_xlim(0, 100)
axes["thin_ice"].set_ylim(0, 700)

axes["thin_ice"].set_xlabel("VELOX thin ice fraction [%]")
axes["thin_ice"].set_ylabel("Falsely-detected CWP [g m$^{-2}$]")

# plot jacobians for cwp and young ice fraction
axes["jacobian"].plot(
    ds_bp.center_freq.sel(channel=da_sen_yif.channel),
    da_sen_yif.mean("time"),
    marker=".",
    label=r"$f_{YI}$=50%",
    color=cmc.batlow(0.3),
)
axes["jacobian"].fill_between(
    ds_bp.center_freq.sel(channel=da_sen_yif.channel),
    da_sen_yif.quantile(0.25, "time"),
    da_sen_yif.quantile(0.75, "time"),
    color=cmc.batlow(0.3),
    linewidth=0,
    alpha=0.5,
)

axes["jacobian"].plot(
    ds_bp.center_freq.sel(channel=da_sen_cwp.channel),
    da_sen_cwp.mean("time"),
    marker=".",
    label=r"$CWP$=150 g m$^{-2}$",
    color=cmc.batlow(0.7),
)
axes["jacobian"].fill_between(
    ds_bp.center_freq.sel(channel=da_sen_cwp.channel),
    da_sen_cwp.quantile(0.25, "time"),
    da_sen_cwp.quantile(0.75, "time"),
    color=cmc.batlow(0.7),
    linewidth=0,
    alpha=0.5,
)

axes["jacobian"].axhline(0, color="k", linestyle="--", linewidth=0.75)

axes["jacobian"].set_xlim(ds_bp.center_freq.min(), ds_bp.center_freq.max())
axes["jacobian"].set_ylim(-5, 20)

axes["jacobian"].legend(frameon=False, loc="center right", bbox_to_anchor=(1, 0.1))

axes["jacobian"].set_xlabel("Frequency [GHz]")
axes["jacobian"].set_ylabel(r"Simulated $\Delta T_b$ [K]")

axes["cdf"].yaxis.set_major_locator(mticker.MultipleLocator(0.2))
axes["cdf"].yaxis.set_minor_locator(mticker.MultipleLocator(0.05))

axes["cdf"].xaxis.set_major_locator(mticker.MultipleLocator(100))
axes["cdf"].xaxis.set_minor_locator(mticker.MultipleLocator(50))
axes["ice_edge"].yaxis.set_major_locator(mticker.MultipleLocator(100))
axes["ice_edge"].yaxis.set_minor_locator(mticker.MultipleLocator(50))
axes["thin_ice"].yaxis.set_major_locator(mticker.MultipleLocator(100))
axes["thin_ice"].yaxis.set_minor_locator(mticker.MultipleLocator(50))

axes["ice_edge"].xaxis.set_major_locator(mticker.MultipleLocator(200))
axes["ice_edge"].xaxis.set_minor_locator(mticker.MultipleLocator(50))

axes["thin_ice"].xaxis.set_major_locator(mticker.MultipleLocator(25))
axes["thin_ice"].xaxis.set_minor_locator(mticker.MultipleLocator(10))

axes["jacobian"].yaxis.set_major_locator(mticker.MultipleLocator(5))
axes["jacobian"].yaxis.set_minor_locator(mticker.MultipleLocator(2.5))

axes["jacobian"].set_xticks([22, 50, 90, 118, 183])
axes["jacobian"].xaxis.set_minor_locator(mticker.MultipleLocator(10))

# colorbar in thin ice fraction plot
plt.draw()
center = (
    axes["thin_ice"].get_position().x0 + axes["thin_ice"].get_position().x1
) / 2
cax = fig.add_axes(
    [
        center - 0.1,
        axes["thin_ice"].get_position().y1 - 0.02,
        0.2,
        0.02,
    ]
)
fig.colorbar(
    im, cax=cax, label=r"$T_{b,90}-T_{b,22}$ [K]", orientation="horizontal"
)

write_figure(
    fig,
    f"paper/fig06.png",
    dpi=300,
    bbox_inches="tight",
)

plt.show()